# Base Model

A simple multilayer ann for classification task of CVEs - in their vectorization format - into their corresponding CWE. Use of PyThorch library

https://machinelearningmastery.com/building-multilayer-perceptron-models-in-pytorch/

To do:
- Define some hiperparameters (number of layers, activation function, learning rate, batch size)
- Are we gonna use some type of regularization like drop out?
- If the real data is not available, we can see if this model is running with some other simple data
- Create some kind of visualization of loss function, accuracy, define the type of metric to use, probably F1-score, accuracy (confusion matrix is maybe too big in this scenario)

In [2]:
# !pip install torch

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 5.2 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 36.4 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 10.3 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 15.5 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 25.2 MB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.5 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 29.5 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 14.8 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 KB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
# !pip install Pinecone

Defaulting to user installation because normal site-packages is not writeable


In [6]:
import torch
import torch.nn as nn

Import data

In [16]:
from pinecone import Pinecone

import numpy as np 

In [5]:
pc = Pinecone(api_key="pcsk_3fsLLS_KMLjRJ9jybj773qMy13aawRBFZveGrn2LrZhVP3GtMWet7PdZgNubyB8xRM6JQ7")
index = pc.Index("mc959")

In [17]:
query_vector = vector = np.random.rand(768).tolist()

# Query the index for the top 5 most similar vectors
response = index.query(
    vector=query_vector,   # Your query vector with 768 dimensions
    top_k=5,               # Retrieve top 5 closest vectors
    include_values=True,   # Include the vector values in the response
    include_metadata=True, # Include any metadata associated with the vectors
)
    
print(response)

{'matches': [{'id': '0',
              'score': -0.0274808984,
              'values': [-0.263852417,
                         -0.0301717371,
                         0.219480604,
                         -0.0583463758,
                         0.351739168,
                         -0.434724867,
                         0.0495806336,
                         0.364792973,
                         0.141762808,
                         -0.101486742,
                         -0.114327475,
                         -0.378488451,
                         -0.121887483,
                         0.142556474,
                         -0.190141588,
                         0.0239429064,
                         0.459951878,
                         0.0719046444,
                         -0.121435225,
                         -0.117923491,
                         0.146112144,
                         -0.203379795,
                         -0.281768709,
                         0.240726605,
       

In [18]:
# Specify the ID of the vector you want to retrieve
entry_id = "9"  # The ID of the specific entry you want to retrieve

# Fetch the vector with the specified ID
response = index.fetch(ids=[entry_id])

# Print the response
print(response)

{'namespace': '',
 'usage': {'read_units': 1},
 'vectors': {'9': {'id': '9',
                   'values': [-0.272559613,
                              0.250883,
                              0.0537423752,
                              0.0189300161,
                              0.199495047,
                              -0.393878222,
                              0.0997849554,
                              0.205802023,
                              0.241373897,
                              -0.01333219,
                              -0.0938622653,
                              -0.187647536,
                              0.0105433082,
                              0.00871945824,
                              -0.199283049,
                              0.148712263,
                              0.455019593,
                              0.124796785,
                              0.0763969,
                              -0.0997588784,
                              0.122478947,
           

In [19]:
# Vector size encoder CVE
V_CVE_size = 768

# Number of possible CWEs to return
N_CWE = 1000

In [8]:
# Model for classification task

model = nn.Sequential()
model.add_module("dense1", nn.Linear(V_CVE_size, 12))
model.add_module("act1", nn.ReLU())
model.add_module("dense2", nn.Linear(12, 8))
model.add_module("act2", nn.ReLU())
model.add_module("output", nn.Linear(8, N_CWE))
model.add_module("outact", nn.Softmax())

In [13]:
print(model)

Sequential(
  (dense1): Linear(in_features=100, out_features=12, bias=True)
  (act1): ReLU()
  (dense2): Linear(in_features=12, out_features=8, bias=True)
  (act2): ReLU()
  (output): Linear(in_features=8, out_features=1000, bias=True)
  (outact): Softmax(dim=None)
)


In [10]:
# Loss function definiton

loss_fn = nn.CrossEntropyLoss()
# loss = loss_fn(output, label)

In [12]:
# Defining the optimization method
Learning_rate = 1e-3

optimizer = torch.optim.Adam(model.parameters(), lr=Learning_rate)

In [ ]:
# Training the model, yet we don't have the X and y

for n in range(num_epochs):
    y_pred = model(X)
    loss = loss_fn(y_pred, y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [14]:
torch.save(model.state_dict(), "base_model.pickle")